## ACL + Semantic Scholar + .bib enhancements

This notebook will fetch the corpus ids of all ACL docs from semantic scholar.

Some titles and other metadata is not accurately extracted by GROBID, so we are populating those by using the .bib files.
The following fields are in the final dataframe that we share -

|   **Column name**  |        **Description**        |
|:------------------:|:-----------------------------:|
|      `acl_id `     |         unique ACL id         |
|     `abstract `    |  abstract extracted by GROBID |
|    `full_text `    | full text extracted by GROBID |
| `corpus_paper_id ` |      Semantic Scholar ID      |
|     `pdf_hash `    |      sha1 hash of the pdf     |
|    `numcitedby `   |  number of citations from S2  |
|       `url `       |      link of publication      |
|    `publisher `    |               -               |
|     `address `     |     Address of conference     |
|       `year`       |               -               |
|      `month `      |               -               |
|    `booktitle `    |               -               |
|      `author `     |        list of authors        |
|      `title `      |         title of paper        |
|      `pages `      |               -               |
|       `doi `       |               -               |
|      `number `     |               -               |
|      `volume `     |               -               |
|     `journal `     |               -               |
|      `editor `     |               -               |
|       `isbn `      |               -               |

In [1]:
import pandas as pd
from pys2 import (
    _get_redshift_connection,
    _load_dataframe_to_redshift,
)
import glob, os

from tqdm import tqdm
import logging
from urllib.parse import urlparse
logging.basicConfig(level=logging.INFO)
tqdm.pandas()

In [2]:
df_acl = pd.read_parquet('../../acl_corpus_full-text.parquet')

In [3]:
df_acl

,acl_id,title,abstract,full_text
0,O02-2002,A Study on Word Similarity using Context Vecto...,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...
1,L02-1310,,,
2,R13-1042,"Headerless, Quoteless, but not Hopeless? Using...",Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...
3,W05-0819,Aligning words in English-Hindi parallel corpora,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al..."
4,L02-1309,,,
...,...,...,...,...
80008,P99-1056,The grapho-phonological system of written Fren...,The processes through which readers evoke ment...,The processes through which readers evoke ment...
80009,P99-1051,Acquiring Lexical Generalizations from Corpora...,This paper examines the extent to which verb d...,This paper examines the extent to which verb d...
80010,P99-1000,Using Aggregation for Selecting Content when G...,"As co-chairs, we had two particular objectives...","As co-chairs, we had two particular objectives..."
80011,P99-1,,,


In [ ]:
_load_dataframe_to_redshift(df_acl['acl_id'].to_frame(), 'public.temp_acl_ids', create_table=True, write_privileges=True)

In [ ]:
query = f"""SELECT cps.corpus_paper_id, cps.source_id, cps.pdf_hash
                        FROM content_ext.paper_sources as cps 
                        join public.temp_acl_ids as tcs
                        on cps.source_id=tcs.acl_id;"""
df_res = pd.read_sql(query, _get_redshift_connection(write_privileges=True))

In [6]:
df_res

,corpus_paper_id,source_id,pdf_hash
0,16975984,W13-3727,eb66e03bea910715f7a82f3600f146a0f2cd022e
1,17493228,W15-3219,be9615d02d46c1b1fa4521f52b210c245f372281
2,18918491,H94-1080,583d605b8c632d130e3779af7205066e2ca78d00
3,184482972,S19-2116,df24ce19ee012bcb642223a430c9ce1fda493220
4,13867055,W18-1816,87941a5597b950d35e104fde4507ec1215c66366
...,...,...,...
73940,222134115,2020.emnlp-main.721,5cda42f13cbb2c4141f41f5f77b83ccd3edf51e9
73941,24010309,L16-1728,318ecd3da4234962764224909504202eb39e7f11
73942,22286897,F13-1017,80cc3c263d3fe99aca0b9fa06c544c484df034c1
73943,174802812,P19-1355,d6a083dad7114f3a39adc65c09bfbb6cf3fee9ea


In [7]:
df_papers_acl = df_acl.merge(df_res, left_on='acl_id', right_on='source_id')
df_papers_acl = df_papers_acl.drop(['source_id'], axis=1)

In [8]:
df_papers_acl.head()

,acl_id,title,abstract,full_text,corpus_paper_id,pdf_hash
0,O02-2002,A Study on Word Similarity using Context Vecto...,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0
1,L02-1310,,,,8220988,8d5e31610bc82c2abc86bc20ceba684c97e66024
2,R13-1042,"Headerless, Quoteless, but not Hopeless? Using...",Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb
3,W05-0819,Aligning words in English-Hindi parallel corpora,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55
4,L02-1309,,,,18078432,011e943b64a78dadc3440674419821ee080f0de3


In [9]:
_load_dataframe_to_redshift(df_papers_acl['corpus_paper_id'].to_frame(), 'public.temp_acl_ids', create_table=True, write_privileges=True)

/net/nfs2.s2-research/shaurya/sw/miniconda3/envs/s2jupy/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
INFO:pys2:Truncating table public.temp_acl_ids
INFO:pys2:Loading new data from S3 into public.temp_acl_ids
INFO:pys2:Done loading public.temp_acl_ids


In [10]:
query = f"""SELECT DISTINCT cps.corpus_paper_id, numcitedby
                        FROM content_ext.paper_es as cps 
                        join public.temp_acl_ids as tcs
                        on cps.corpus_paper_id=tcs.corpus_paper_id;"""
df_res = pd.read_sql(query, _get_redshift_connection(write_privileges=True))

/net/nfs2.s2-research/shaurya/sw/miniconda3/envs/s2jupy/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
df_res = df_res.drop_duplicates()
df_res = df_res.groupby(['corpus_paper_id'], sort=False)['numcitedby'].max().reset_index()

S2 stores all the citations of the paper over the years. We just need the latest number

In [12]:
df_papers_acl = df_papers_acl.merge(df_res, on='corpus_paper_id')

In [13]:
df_papers_acl.head()

,acl_id,title,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby
0,O02-2002,A Study on Word Similarity using Context Vecto...,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0,14
1,L02-1310,,,,8220988,8d5e31610bc82c2abc86bc20ceba684c97e66024,93
2,R13-1042,"Headerless, Quoteless, but not Hopeless? Using...",Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb,10
3,W05-0819,Aligning words in English-Hindi parallel corpora,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55,15
4,L02-1309,,,,18078432,011e943b64a78dadc3440674419821ee080f0de3,12


# Adding missing titles, years and abstracts from .bib

In [14]:
import bibtexparser

list_acl_bib = []
for file in tqdm(glob.glob('ACL/bib/*.bib')):
    with open(file) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)
        try:
            _dict =  bib_database.entries[0]
            _dict['acl_id'] = os.path.basename(file)[:-4]
            list_acl_bib.append(_dict)
        except:
            continue
df_bib = pd.DataFrame(list_acl_bib)

100%|██████████| 80014/80014 [14:14<00:00, 93.59it/s] 


In [15]:
df_bib.head()

,url,publisher,address,year,month,booktitle,author,title,ENTRYTYPE,ID,...,abstract,pages,doi,number,volume,journal,editor,isbn,language,note
0,http://www.lrec-conf.org/proceedings/lrec2004/...,European Language Resources Association (ELRA),"Lisbon, Portugal",2004,May,Proceedings of the Fourth International Confer...,"Traum, David R. and\nRobinson, Susan and\nSt...",Evaluation of Multi-party Virtual Reality Dial...,inproceedings,traum-etal-2004-evaluation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://aclanthology.org/D19-5024,Association for Computational Linguistics,"Hong Kong, China",2019,November,Proceedings of the Second Workshop on Natural ...,"Da San Martino, Giovanni and\nBarr{\'o}n-Cede...",Findings of the {NLP}4{IF}-2019 Shared Task on...,inproceedings,da-san-martino-etal-2019-findings,...,We present the shared task on Fine-Grained Pro...,162--170,10.18653/v1/D19-5024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://aclanthology.org/P82-1017,Association for Computational Linguistics,"Toronto, Ontario, Canada",1982,June,20th Annual Meeting of the Association for Com...,"Walker, Donald E.",Reflections on 20 Years of the {ACL}: An Intro...,inproceedings,walker-1982-reflections,...,NaN,89--91,10.3115/981251.981273,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://aclanthology.org/2020.emnlp-main.50,Association for Computational Linguistics,Online,2020,November,Proceedings of the 2020 Conference on Empirica...,"Li, Manling and\nZeng, Qi and\nLin, Ying an...",Connecting the Dots: Event Graph Schema Induct...,inproceedings,li-etal-2020-connecting,...,Event schemas can guide our understanding and ...,684--695,10.18653/v1/2020.emnlp-main.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://aclanthology.org/C80-1091,NaN,NaN,1980,NaN,{COLING} 1980 Volume 1: The 8th International ...,"Tuldava, Juhan",A Mathematical Model of the Vocabulary-Text Re...,inproceedings,tuldava-1980-mathematical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_papers_acl = df_papers_acl.merge(df_bib, on='acl_id')

In [17]:
df_papers_acl = df_papers_acl.drop(['title_x', 'abstract_y', 'ENTRYTYPE', 'ID', 'language', 'note'], axis=1)

In [18]:
df_papers_acl = df_papers_acl.rename(columns={"title_y": "title", "abstract_x": "abstract"})

In [19]:
df_papers_acl.columns

Index(['acl_id', 'abstract', 'full_text', 'corpus_paper_id', 'pdf_hash',
       'numcitedby', 'url', 'publisher', 'address', 'year', 'month',
       'booktitle', 'author', 'title', 'pages', 'doi', 'number', 'volume',
       'journal', 'editor', 'isbn'],
      dtype='object')

In [20]:
df_papers_acl.head()

,acl_id,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby,url,publisher,address,year,...,booktitle,author,title,pages,doi,number,volume,journal,editor,isbn
0,O02-2002,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0,14,https://aclanthology.org/O02-2002,NaN,NaN,2002,...,International Journal of Computational Linguis...,"Chen, Keh-Jiann and\nYou, Jia-Ming",A Study on Word Similarity using Context Vecto...,37--58,NaN,NaN,NaN,NaN,NaN,NaN
1,L02-1310,,,8220988,8d5e31610bc82c2abc86bc20ceba684c97e66024,93,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Mihalcea, Rada F.",Bootstrapping Large Sense Tagged Corpora,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R13-1042,Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb,10,https://aclanthology.org/R13-1042,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,Proceedings of the International Conference Re...,"Jamison, Emily and\nGurevych, Iryna","Headerless, Quoteless, but not Hopeless? Using...",327--335,NaN,NaN,NaN,NaN,NaN,NaN
3,W05-0819,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55,15,https://aclanthology.org/W05-0819,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,Proceedings of the {ACL} Workshop on Building ...,"Aswani, Niraj and\nGaizauskas, Robert",Aligning Words in {E}nglish-{H}indi Parallel C...,115--118,NaN,NaN,NaN,NaN,NaN,NaN
4,L02-1309,,,18078432,011e943b64a78dadc3440674419821ee080f0de3,12,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,Proceedings of the Third International Confere...,"Suyaga, Fumiaki and\nTakezawa, Toshiyuki and...",Proposal of a very-large-corpus acquisition me...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_papers_acl.to_parquet('acl-publication-info.74k.parquet', compression='gzip')